In [6]:
#load the example files I have

import os 
import json
samples = list()

folder = "my_examples"

for _file in os.listdir(folder):
    filepath = os.path.join(folder, _file)
    with open(filepath, 'rb') as f:
        data = json.load(f)
        _input ={
            "primarySourceText": data["primarySourceText"],
            "citation": data["citation"]
        }
        samples.append({
            "input": _input, 
            "output": data
        })
samples[0]

my_examples\aeolus.json
my_examples\hydra.json
my_examples\lotus.json


{'input': {'primarySourceText': 'Then to the Aeolian isle we came, where dwelt Aeolus, son of Hippotas, dear to the immortal gods, in a floating island, and all around it is a wall of unbreakable bronze, and the cliff runs up sheer. [5] Twelve children of his, too, there are in the halls, six daughters and six sturdy sons, and he gave his daughters to his sons to wife. These, then, feast continually by their dear father and good mother, and before them lies boundless good cheer. [10] And the house, filled with the savour of feasting, resounds all about even in the outer court by day,1 and by night again they sleep beside their chaste wives on blankets and on corded bedsteads. To their city, then, and fair palace did we come, and for a full month he made me welcome and questioned me about each thing, [15] about Ilios, and the ships of the Argives, and the return of the Achaeans. And I told him all the tale in due order. But when I, on my part, asked him that I might depart and bade him 

In [51]:
import openai 
import asyncio
openai.api_key = input("what is gpt api key")
model_engine = "gpt-3.5-turbo"

general_prompt = f"""You are a game designer assistance for me who is trying to create an interactive Odyssey game. 
Given primary text, you are to evolve it into game components that I can process and feed to the user.
Each primary text is linked to a Node. A node is made up of a primarySourceText, a citation (most provided) as well as an entranceDescription (what people see before encountering anyone, and component (a list of components).

Each component represents a seperate part of the island and has the following fields: status (friendly/nuetral/hostile), fightingStrength: 0-5, intelligence: 0-5, agression: 0-5, explorerDescription: what you see right before encountering this creature, exhibitsXenia: if this creature will feed and house the character; who: what this creature is; fightingDetails: any associated fighting details; sampleBenefits: a mapper for each quality of food/fame/gold/shipQaulity to an amount of how it increases and an example of how it happens.

Here are some examples:
""" + "\n\n\n__________________________\n\n\n".join(["input:\n"+str(s["input"]) + "\noutput:\n" + str(s["output"]) for s in samples]) + "\n\n\nRemember, the response should be JSON serializable. Use double quotes everywhere."
general_prompt

'You are a game designer assistance for me who is trying to create an interactive Odyssey game. \nGiven primary text, you are to evolve it into game components that I can process and feed to the user.\nEach primary text is linked to a Node. A node is made up of a primarySourceText, a citation (most provided) as well as an entranceDescription (what people see before encountering anyone, and component (a list of components).\n\nEach component represents a seperate part of the island and has the following fields: status (friendly/nuetral/hostile), fightingStrength: 0-5, intelligence: 0-5, agression: 0-5, explorerDescription: what you see right before encountering this creature, exhibitsXenia: if this creature will feed and house the character; who: what this creature is; fightingDetails: any associated fighting details; sampleBenefits: a mapper for each quality of food/fame/gold/shipQaulity to an amount of how it increases and an example of how it happens.\n\nHere are some examples:\ninpu

In [52]:
#get input files for gpt
_inputs = list()
folder = "primary_sources_for_gpt"
input_list = os.listdir(folder)
for _file in input_list:
    filepath = os.path.join(folder, _file)
    with open(filepath, 'rb') as f:
        d = json.load(f)
        _inputs.append(d)


#get completions for each of these
completions = list()
for specific_input in _inputs:
    print("input:\n"+str(specific_input))
    completion = openai.ChatCompletion.acreate(
                model = model_engine,
                messages = [{"role": "system", "content": general_prompt}] +[{"role": "user", "content": "input:\n"+str(specific_input)}]
                            
            )
    completions.append(completion)

input:
{'primarySourceText': 'I presently saw smoke and a great billow, and heard a booming. Then from the hands of my men in their terror the oars flew, and splashed one and all in the swirl, and [205] the ship stood still where it was, when they no longer plied with their hands the tapering oars. But I went through the ship and cheered my men with gentle words, coming up to each man in turn: “‘Friends, hitherto we have been in no wise ignorant of sorrow; surely this evil that besets us now is no greater than when the Cyclops [210] penned us in his hollow cave by brutal strength; yet even thence we made our escape through my valor and counsel and wit; these dangers, too, methinks we shall some day remember. But now come, as I bid, let us all obey. Do you keep your seats on the benches [215] and smite with your oars the deep surf of the sea, in the hope that Zeus may grant us to escape and avoid this death. And to thee, steersman, I give this command, and do thou lay it to heart, since

In [53]:
completions = [await c for c in completions]
completions

[<OpenAIObject chat.completion id=chatcmpl-99Hf0W243oSuT3ZS3fiUmAOyYpDI0 at 0x1d1ba3f3860> JSON: {
   "id": "chatcmpl-99Hf0W243oSuT3ZS3fiUmAOyYpDI0",
   "object": "chat.completion",
   "created": 1711999866,
   "model": "gpt-3.5-turbo-0125",
   "choices": [
     {
       "index": 0,
       "message": {
         "role": "assistant",
         "content": "{\n  \"primarySourceText\": \"I presently saw smoke and a great billow, and heard a booming. Then from the hands of my men in their terror the oars flew, and splashed one and all in the swirl, and [205] the ship stood still where it was, when they no longer plied with their hands the tapering oars. But I went through the ship and cheered my men with gentle words, coming up to each man in turn: \\u201c\\u2018Friends, hitherto we have been in no wise ignorant of sorrow; surely this evil that besets us now is no greater than when the Cyclops [210] penned us in his hollow cave by brutal strength; yet even thence we made our escape through my

In [54]:
# json.loads(completions[0].choices[0].message.content)

output_folder = "gpt_examples" 

ignore_names = ["charybdis_scylla.json", "lamus.json", "sirens.json"]
for idx, filename in enumerate(input_list):
    try:
        this_content = json.loads(completions[idx].choices[0].message.content)
        print("good", completions[idx].choices[0].message.content)
    except:
        print("bad", completions[idx].choices[0].message.content)
        continue
        
    with open(os.path.join(output_folder, filename), 'w') as f:
        json.dump(this_content, f)

good {
  "primarySourceText": "I presently saw smoke and a great billow, and heard a booming. Then from the hands of my men in their terror the oars flew, and splashed one and all in the swirl, and [205] the ship stood still where it was, when they no longer plied with their hands the tapering oars. But I went through the ship and cheered my men with gentle words, coming up to each man in turn: \u201c\u2018Friends, hitherto we have been in no wise ignorant of sorrow; surely this evil that besets us now is no greater than when the Cyclops [210] penned us in his hollow cave by brutal strength; yet even thence we made our escape through my valor and counsel and wit; these dangers, too, methinks we shall some day remember. But now come, as I bid, let us all obey. Do you keep your seats on the benches [215] and smite with your oars the deep surf of the sea, in the hope that Zeus may grant us to escape and avoid this death. And to thee, steersman, I give this command, and do thou lay it to h